In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


# from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.ensemble import GradientBoostingRegressor


import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics as st
import seaborn as sns

from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

from IPython.display import display, HTML

In [4]:
def split_headlines(df):

    patterns = ['buy_now', 'shop_now', 'limited_offer', 'add_cart', 
                'use_code', 'free_shipping', 'today_only', 'free_delivery', 
                'extra_discount', 'voucher_code', 'last_chance', 'up_to', 
                '%_off']

    for i in patterns:
        pattern_split = i.split('_')    
        pattern = re.compile(rf"\b{pattern_split[0]}.*\b{pattern_split[1]}", 
                             re.I)
        df[i] = df.apply(lambda x: 1 if pattern.search(x.headline) else 0, 
                         axis=1)
        
    return df

In [5]:
df = pd.read_csv('fb_19_20.csv')

In [6]:
df = split_headlines(df)

In [7]:
df.head()

,date,country,brand_name,channel,profile_name,campaign_name,ad_set_name,ad_name,headline,cost,...,add_cart,use_code,free_shipping,today_only,free_delivery,extra_discount,voucher_code,last_chance,up_to,%_off
0,2019-11-24,PH,Enfa,Lazada,PH - Enfa - Facebook,MEAD-ECOMMERCE-2019-13 - Enfa Always On Q4,AO Q4 - Prospecting,AO Q4 - Enfa - Milk Guarantee,Buy Enfagrow A+ Four at Lazada Now!,24.180000,...,0,0,0,0,0,0,0,0,0,0
1,2019-11-25,PH,Enfa,Lazada,PH - Enfa - Facebook,MEAD-ECOMMERCE-2019-13 - Enfa Always On Q4,AO Q4 - Prospecting,AO Q4 - Enfa - Milk Guarantee,Buy Enfagrow A+ Four at Lazada Now!,23.700001,...,0,0,0,0,0,0,0,0,0,0
2,2019-11-26,PH,Enfa,Lazada,PH - Enfa - Facebook,MEAD-ECOMMERCE-2019-13 - Enfa Always On Q4,AO Q4 - Prospecting,AO Q4 - Enfa - Milk Guarantee,Buy Enfagrow A+ Four at Lazada Now!,23.910000,...,0,0,0,0,0,0,0,0,0,0
3,2019-11-24,PH,Enfa,Lazada,PH - Enfa - Facebook,MEAD-ECOMMERCE-2019-13 - Enfa Always On Q4,AO Q4 - Prospecting CV,AO Q4 - Enfa - Milk Guarantee,Buy Enfagrow A+ Four at Lazada Now!,18.559999,...,0,0,0,0,0,0,0,0,0,0
4,2019-11-25,PH,Enfa,Lazada,PH - Enfa - Facebook,MEAD-ECOMMERCE-2019-13 - Enfa Always On Q4,AO Q4 - Prospecting CV,AO Q4 - Enfa - Milk Guarantee,Buy Enfagrow A+ Four at Lazada Now!,21.250000,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.columns

Index(['date', 'country', 'brand_name', 'channel', 'profile_name',
       'campaign_name', 'ad_set_name', 'ad_name', 'headline', 'cost',
       'impressions', 'clicks', 'ctr', 'ave_cpc', 'link_clicks',
       'outbound_clicks', 'view_content_actions', 'add_cart_actions',
       'add_cart_value', 'purchase_actions', 'purchase_value', 'buy_now',
       'shop_now', 'limited_offer', 'add_cart', 'use_code', 'free_shipping',
       'today_only', 'free_delivery', 'extra_discount', 'voucher_code',
       'last_chance', 'up_to', '%_off'],
      dtype='object')

In [9]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [27]:
df_model = df[['brand_name', 'channel', 'cost', 'buy_now', 'shop_now', 'limited_offer', 'add_cart', 'use_code', 'free_shipping', 'today_only',
             'free_delivery', 'extra_discount', 'voucher_code', 'last_chance', 'up_to', '%_off'] + ['impressions']]
df_model

,brand_name,channel,cost,buy_now,shop_now,limited_offer,add_cart,use_code,free_shipping,today_only,free_delivery,extra_discount,voucher_code,last_chance,up_to,%_off,impressions
0,Enfa,Lazada,24.180000,1,0,0,0,0,0,0,0,0,0,0,0,0,41794
1,Enfa,Lazada,23.700001,1,0,0,0,0,0,0,0,0,0,0,0,0,42927
2,Enfa,Lazada,23.910000,1,0,0,0,0,0,0,0,0,0,0,0,0,25883
3,Enfa,Lazada,18.559999,1,0,0,0,0,0,0,0,0,0,0,0,0,30976
4,Enfa,Lazada,21.250000,1,0,0,0,0,0,0,0,0,0,0,0,0,31711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11206,Sustagen,Shopee,15.890000,1,0,0,0,0,0,0,0,0,0,0,1,1,5346
11207,Sustagen,Shopee,8.910000,1,0,0,0,0,0,0,0,0,0,0,1,1,3064
11208,Sustagen,Shopee,0.280000,1,0,0,0,0,0,0,0,0,0,0,1,1,26
11209,Sustagen,Shopee,7.990000,1,0,0,0,0,0,0,0,0,0,0,1,1,2962


In [28]:
df_model_dummies = pd.get_dummies(df_model, columns=['brand_name', 'channel'])
df_model_dummies

,cost,buy_now,shop_now,limited_offer,add_cart,use_code,free_shipping,today_only,free_delivery,extra_discount,voucher_code,last_chance,up_to,%_off,impressions,brand_name_Enfa,brand_name_Lactum,brand_name_Sustagen,channel_Lazada,channel_Shopee
0,24.180000,1,0,0,0,0,0,0,0,0,0,0,0,0,41794,1,0,0,1,0
1,23.700001,1,0,0,0,0,0,0,0,0,0,0,0,0,42927,1,0,0,1,0
2,23.910000,1,0,0,0,0,0,0,0,0,0,0,0,0,25883,1,0,0,1,0
3,18.559999,1,0,0,0,0,0,0,0,0,0,0,0,0,30976,1,0,0,1,0
4,21.250000,1,0,0,0,0,0,0,0,0,0,0,0,0,31711,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11206,15.890000,1,0,0,0,0,0,0,0,0,0,0,1,1,5346,0,0,1,0,1
11207,8.910000,1,0,0,0,0,0,0,0,0,0,0,1,1,3064,0,0,1,0,1
11208,0.280000,1,0,0,0,0,0,0,0,0,0,0,1,1,26,0,0,1,0,1
11209,7.990000,1,0,0,0,0,0,0,0,0,0,0,1,1,2962,0,0,1,0,1


In [29]:
df_target = df_model_dummies['impressions']
df_target

0        41794
1        42927
2        25883
3        30976
4        31711
         ...  
11206     5346
11207     3064
11208       26
11209     2962
11210     1204
Name: impressions, Length: 11211, dtype: int64

In [30]:
df_features = df_model_dummies.drop(['impressions'], axis=1)
df_features

,cost,buy_now,shop_now,limited_offer,add_cart,use_code,free_shipping,today_only,free_delivery,extra_discount,voucher_code,last_chance,up_to,%_off,brand_name_Enfa,brand_name_Lactum,brand_name_Sustagen,channel_Lazada,channel_Shopee
0,24.180000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,23.700001,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,23.910000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,18.559999,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,21.250000,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11206,15.890000,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1
11207,8.910000,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1
11208,0.280000,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1
11209,7.990000,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1


In [31]:
X_train_0_1, X_test_0_1, y_train_0_1, y_test_0_1 = train_test_split(df_features, df_target)

In [32]:
#For purpose of Demonstration let's just use the default setting of the following models
LR = LinearRegression()
LR.fit(X_train_0_1, y_train_0_1)
Lasso1 = Lasso()
Lasso1.fit(X_train_0_1, y_train_0_1)
Ridge1 = Ridge()
Ridge1.fit(X_train_0_1, y_train_0_1)
#kNN = KNeighborsRegressor(n_neighbors=8) 
kNN = KNeighborsRegressor() 
kNN.fit(X_train_0_1,y_train_0_1)
#DT = DecisionTreeRegressor(max_depth=5) 
DT = DecisionTreeRegressor() 
DT.fit(X_train_0_1,y_train_0_1)
#RF = RandomForestRegressor(n_estimators = 100) 
RF = RandomForestRegressor() 
RF.fit(X_train_0_1,y_train_0_1)
#GBM = GradientBoostingRegressor(max_depth =5, learning_rate=.1) 
GBM = GradientBoostingRegressor() 
GBM.fit(X_train_0_1,y_train_0_1)

# y_test_LR = LR.predict(X_test)
# y_test_Lasso = Lasso.predict(X_test)
# y_test_Ridge = Ridge1.predict(X_test)
# y_test_kNN = kNN.predict(X_test)
# y_test_DT = DT.predict(X_test)
# y_test_RF = RF.predict(X_test)
# y_test_GBM = GBM.predict(X_test)


cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[0] = ['Linear Regression', LR.score(X_train_0_1, y_train_0_1), LR.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[1] = ['Linear Regression + Lasso', Lasso1.score(X_train_0_1, y_train_0_1), Lasso1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[2] = ['Linear Regression + Ridge', Ridge1.score(X_train_0_1, y_train_0_1), Ridge1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

C:\Users\Jelo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1077605566073.7344, tolerance: 19908634723.072178
  positive)


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.649599,0.590225
1,Linear Regression + Lasso,0.649599,0.590238
2,Linear Regression + Ridge,0.649597,0.590315
3,kNN,0.743999,0.636528
4,Decision Tree,0.999238,0.378824
5,Random Forest,0.955954,0.659607
6,Gradient Boosting Method,0.824726,0.654347


In [19]:
est = GradientBoostingRegressor()
param_grids = {'learning_rate': [.2, 0.1, 0.05, 0.02, 0.01],
              'max_depth': [3, 4, 6, 10, 14],
              'min_samples_leaf': [2, 3, 4],
              'max_features': [.5,.3, .2] 
}   
gs_cv = GridSearchCV(est, param_grids).fit(X_train_0_1, y_train_0_1)
print(gs_cv.best_params_)

{'learning_rate': 0.02, 'max_depth': 10, 'max_features': 0.2, 'min_samples_leaf': 3}


In [36]:
GBM = GradientBoostingRegressor(learning_rate = 0.02, max_depth=10, max_features=0.2, min_samples_leaf=3) 
GBM.fit(X_train_0_1,y_train_0_1)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.02, loss='ls',
                          max_depth=10, max_features=0.2, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [34]:
GBM = GradientBoostingRegressor() 
GBM.fit(X_train_0_1,y_train_0_1)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [37]:
print(GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1))

0.8326329935262811 0.7435625016539056


In [16]:
targets = ['impressions', 'clicks', 'ctr', 'ave_cpc', 'link_clicks',
           'outbound_clicks', 'view_content_actions', 'add_cart_actions',
           'add_cart_value', 'purchase_actions', 'purchase_value']

for i in targets:
    df_model = df[['brand_name', 'channel', 'cost', 'buy_now', 'shop_now', 'limited_offer', 
                   'add_cart', 'use_code', 'free_shipping', 'today_only',
                   'free_delivery', 'extra_discount', 'voucher_code', 'last_chance', 
                   'up_to', '%_off'] + [i]]
    
    df_model_dummies = pd.get_dummies(df_model, columns=['brand_name', 'channel'])
    df_target = df_model_dummies[i]
    df_features = df_model_dummies.drop([i], axis=1)
    X_train_0_1, X_test_0_1, y_train_0_1, y_test_0_1 = train_test_split(df_features, df_target)
    
    #For purpose of Demonstration let's just use the default setting of the following models
    LR = LinearRegression()
    LR.fit(X_train_0_1, y_train_0_1)
    Lasso1 = Lasso()
    Lasso1.fit(X_train_0_1, y_train_0_1)
    Ridge1 = Ridge()
    Ridge1.fit(X_train_0_1, y_train_0_1)
    #kNN = KNeighborsRegressor(n_neighbors=8) 
    kNN = KNeighborsRegressor() 
    kNN.fit(X_train_0_1,y_train_0_1)
    #DT = DecisionTreeRegressor(max_depth=5) 
    DT = DecisionTreeRegressor() 
    DT.fit(X_train_0_1,y_train_0_1)
    #RF = RandomForestRegressor(n_estimators = 100) 
    RF = RandomForestRegressor() 
    RF.fit(X_train_0_1,y_train_0_1)
    #GBM = GradientBoostingRegressor(max_depth =5, learning_rate=.1) 
    GBM = GradientBoostingRegressor() 
    GBM.fit(X_train_0_1,y_train_0_1)

    # y_test_LR = LR.predict(X_test)
    # y_test_Lasso = Lasso.predict(X_test)
    # y_test_Ridge = Ridge1.predict(X_test)
    # y_test_kNN = kNN.predict(X_test)
    # y_test_DT = DT.predict(X_test)
    # y_test_RF = RF.predict(X_test)
    # y_test_GBM = GBM.predict(X_test)


    cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
    df_0_1 = pd.DataFrame(columns=cols)

    df_0_1.loc[0] = ['Linear Regression', LR.score(X_train_0_1, y_train_0_1), LR.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[1] = ['Linear Regression + Lasso', Lasso1.score(X_train_0_1, y_train_0_1), Lasso1.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[2] = ['Linear Regression + Ridge', Ridge1.score(X_train_0_1, y_train_0_1), Ridge1.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
    df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

    
    print(i)    
    display(df_0_1)

C:\Users\Jelo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1015773584668.2344, tolerance: 19063485020.381737
  positive)


impressions


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.646790,0.602246
1,Linear Regression + Lasso,0.646790,0.602252
2,Linear Regression + Ridge,0.646789,0.602271
3,kNN,0.752637,0.635165
4,Decision Tree,0.999260,0.453957
5,Random Forest,0.956360,0.704341
6,Gradient Boosting Method,0.810507,0.639618


clicks


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.640685,0.576169
1,Linear Regression + Lasso,0.639964,0.576815
2,Linear Regression + Ridge,0.640681,0.576221
3,kNN,0.760516,0.679019
4,Decision Tree,0.999218,0.576158
5,Random Forest,0.967522,0.742336
6,Gradient Boosting Method,0.838598,0.757187


...

,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.147386,0.184782
1,Linear Regression + Lasso,0.146830,0.184418
2,Linear Regression + Ridge,0.147384,0.184793
3,kNN,0.432619,0.041263
4,Decision Tree,0.993296,-0.184752
5,Random Forest,0.889951,0.169690
6,Gradient Boosting Method,0.461080,0.225923


purchase_actions


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.173734,0.195839
1,Linear Regression + Lasso,0.121424,0.147771
2,Linear Regression + Ridge,0.173733,0.195828
3,kNN,0.452229,0.088859
4,Decision Tree,0.977215,-0.376341
5,Random Forest,0.869997,0.140833
6,Gradient Boosting Method,0.516455,0.272685


purchase_value


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.093257,0.169961
1,Linear Regression + Lasso,0.091001,0.168293
2,Linear Regression + Ridge,0.093256,0.170012
3,kNN,0.347493,-0.048658
4,Decision Tree,0.994928,-1.858022
5,Random Forest,0.865075,0.011932
6,Gradient Boosting Method,0.480015,0.002243


In [38]:
df_model = df[['brand_name', 'channel', 'cost'] + ['impressions']]
df_model

,brand_name,channel,cost,impressions
0,Enfa,Lazada,24.180000,41794
1,Enfa,Lazada,23.700001,42927
2,Enfa,Lazada,23.910000,25883
3,Enfa,Lazada,18.559999,30976
4,Enfa,Lazada,21.250000,31711
...,...,...,...,...
11206,Sustagen,Shopee,15.890000,5346
11207,Sustagen,Shopee,8.910000,3064
11208,Sustagen,Shopee,0.280000,26
11209,Sustagen,Shopee,7.990000,2962


In [39]:
df_model_dummies = pd.get_dummies(df_model, columns=['brand_name', 'channel'])
df_model_dummies

,cost,impressions,brand_name_Enfa,brand_name_Lactum,brand_name_Sustagen,channel_Lazada,channel_Shopee
0,24.180000,41794,1,0,0,1,0
1,23.700001,42927,1,0,0,1,0
2,23.910000,25883,1,0,0,1,0
3,18.559999,30976,1,0,0,1,0
4,21.250000,31711,1,0,0,1,0
...,...,...,...,...,...,...,...
11206,15.890000,5346,0,0,1,0,1
11207,8.910000,3064,0,0,1,0,1
11208,0.280000,26,0,0,1,0,1
11209,7.990000,2962,0,0,1,0,1


In [40]:
df_target = df_model_dummies['impressions']
df_target

0        41794
1        42927
2        25883
3        30976
4        31711
         ...  
11206     5346
11207     3064
11208       26
11209     2962
11210     1204
Name: impressions, Length: 11211, dtype: int64

In [41]:
df_features = df_model_dummies.drop(['impressions'], axis=1)
df_features

,cost,brand_name_Enfa,brand_name_Lactum,brand_name_Sustagen,channel_Lazada,channel_Shopee
0,24.180000,1,0,0,1,0
1,23.700001,1,0,0,1,0
2,23.910000,1,0,0,1,0
3,18.559999,1,0,0,1,0
4,21.250000,1,0,0,1,0
...,...,...,...,...,...,...
11206,15.890000,0,0,1,0,1
11207,8.910000,0,0,1,0,1
11208,0.280000,0,0,1,0,1
11209,7.990000,0,0,1,0,1


In [42]:
X_train_0_1, X_test_0_1, y_train_0_1, y_test_0_1 = train_test_split(df_features, df_target)

In [43]:
#For purpose of Demonstration let's just use the default setting of the following models
LR = LinearRegression()
LR.fit(X_train_0_1, y_train_0_1)
Lasso1 = Lasso()
Lasso1.fit(X_train_0_1, y_train_0_1)
Ridge1 = Ridge()
Ridge1.fit(X_train_0_1, y_train_0_1)
#kNN = KNeighborsRegressor(n_neighbors=8) 
kNN = KNeighborsRegressor() 
kNN.fit(X_train_0_1,y_train_0_1)
#DT = DecisionTreeRegressor(max_depth=5) 
DT = DecisionTreeRegressor() 
DT.fit(X_train_0_1,y_train_0_1)
#RF = RandomForestRegressor(n_estimators = 100) 
RF = RandomForestRegressor() 
RF.fit(X_train_0_1,y_train_0_1)
#GBM = GradientBoostingRegressor(max_depth =5, learning_rate=.1) 
GBM = GradientBoostingRegressor() 
GBM.fit(X_train_0_1,y_train_0_1)

# y_test_LR = LR.predict(X_test)
# y_test_Lasso = Lasso.predict(X_test)
# y_test_Ridge = Ridge1.predict(X_test)
# y_test_kNN = kNN.predict(X_test)
# y_test_DT = DT.predict(X_test)
# y_test_RF = RF.predict(X_test)
# y_test_GBM = GBM.predict(X_test)


cols = ['Regression Method','Train Accuracy', 'Test Accuracy']
df_0_1 = pd.DataFrame(columns=cols)

df_0_1.loc[0] = ['Linear Regression', LR.score(X_train_0_1, y_train_0_1), LR.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[1] = ['Linear Regression + Lasso', Lasso1.score(X_train_0_1, y_train_0_1), Lasso1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[2] = ['Linear Regression + Ridge', Ridge1.score(X_train_0_1, y_train_0_1), Ridge1.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[3] = ['kNN', kNN.score(X_train_0_1, y_train_0_1), kNN.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[4] = ['Decision Tree',DT.score(X_train_0_1, y_train_0_1), DT.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[5] = ['Random Forest',RF.score(X_train_0_1, y_train_0_1), RF.score(X_test_0_1, y_test_0_1)]
df_0_1.loc[6] = ['Gradient Boosting Method',GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1)]

df_0_1

C:\Users\Jelo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1198923928075.0234, tolerance: 20596919267.302505
  positive)


,Regression Method,Train Accuracy,Test Accuracy
0,Linear Regression,0.623756,0.597752
1,Linear Regression + Lasso,0.623756,0.597751
2,Linear Regression + Ridge,0.623756,0.597751
3,kNN,0.737389,0.593670
4,Decision Tree,0.998347,0.357590
5,Random Forest,0.940311,0.576443
6,Gradient Boosting Method,0.793159,0.637868


In [44]:
est = GradientBoostingRegressor()
param_grids = {'learning_rate': [.2, 0.1, 0.05, 0.02, 0.01],
              'max_depth': [3, 4, 6, 10, 14],
              'min_samples_leaf': [2, 3, 4],
              'max_features': [.5,.3, .2] 
}   
gs_cv = GridSearchCV(est, param_grids).fit(X_train_0_1, y_train_0_1)
print(gs_cv.best_params_)

{'learning_rate': 0.1, 'max_depth': 3, 'max_features': 0.3, 'min_samples_leaf': 3}


In [45]:
GBM = GradientBoostingRegressor(learning_rate = 0.1, max_depth=3, max_features=0.3, min_samples_leaf=3) 
GBM.fit(X_train_0_1,y_train_0_1)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=0.3, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [46]:
print(GBM.score(X_train_0_1, y_train_0_1), GBM.score(X_test_0_1, y_test_0_1))

0.7267162853802068 0.6858861522829283
